In [0]:
# 00_validate_bronze_providers.ipynb
# SOURCE: Bronze Providers table
# OUTPUT: N/A (side‑validation)

from pyspark.sql import functions as F

BRONZE_PATH = "dbfs:/kardia/bronze/bronze_providers"
df = spark.read.format("delta").load(BRONZE_PATH)

In [0]:
# Check for NULLs and uniqueness after Bronze ingestion.
stats = df.agg(
    F.count("*").alias("row_count"),
    F.countDistinct("ProviderID").alias("distinct_ids"),
    F.sum(F.when(F.col("ProviderID").isNull(), 1).otherwise(0)).alias("null_ids"),
    F.sum(F.when(F.col("ProviderSpecialty").isNull(), 1).otherwise(0)).alias("null_specialty"),
    F.sum(F.when(F.col("ProviderLocation").isNull(), 1).otherwise(0)).alias("null_location")
).first()
print("Validation stats:", stats.asDict())

assert stats.null_ids == 0, f"{stats.null_ids} null ProviderID(s)"
assert stats.distinct_ids == stats.row_count, "Duplicate ProviderID(s) found"
print("Bronze Providers validation passed")